In [39]:
import pandas as pd
import requests
import nltk
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('de_core_news_sm')
from bs4 import BeautifulSoup

# Scrape the table data

In [40]:
URL = "https://eur-lex.europa.eu/legal-content/DE/TXT/HTML/?uri=CELEX:02012R0432-20210517#tocId5"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [41]:
# Find the <p> tag with class "title-table"
title_table_p = soup.find('p', class_='title-table')

# Find the next <table> tag following the title-table <p> tag
desired_table = title_table_p.find_next('table')

# Replace sub-tables in cells with a placeholder string
for cell in desired_table.find_all('td'):
    sub_table = cell.find('table')
    if sub_table:
        cell.string = 'SUB_TABLE_PLACEHOLDER'

# Format and clean the data

In [42]:
headers = [header.get_text(strip=True) for header in desired_table.find('tr').find_all('td')]

In [43]:
df = pd.read_html(str(desired_table), header=0)[0]
df = df.drop(columns=['Bedingungen und/oder Beschränkungen hinsichtlich der Verwendung des Lebensmittels und/oder zusätzliche Erklärungen oder Warnungen',
                       'Nummer im EFSA Journal',
                       'Nummer des Eintrags in der konsolidierten Liste, die der EFSA zur Bewertung vorgelegt wurde'])
df = df.rename(columns={"Nährstoff, Substanz, Lebensmittel oder Lebensmittelkategorie": "Nährstoff",
                        "Angabe":"Claim",
                   "Bedingungen für die Verwendung der Angabe": "Bedingungen",
                  })

In [44]:
# Define the pattern to look for in the specified columns
pattern_to_drop = "▼"

# Create a boolean mask indicating which rows to drop
rows_to_drop = df.apply(lambda row: any(pattern_to_drop in value for value in row), axis=1)

# Drop the rows from the DataFrame
df = df[~rows_to_drop]

# Process the claims

Stemming: Turn them into their basic form (walk instead of walking, tree instead of trees, ...)

Remove stop words: Stop words are words like "and", "or", etc. which are not important for the meaning of a sentence


In [45]:
# Function to apply stemming to a text
def stem_text(text):
    clean_text = [word.lemma_ for word in nlp(text) if not word.is_stop and word.is_alpha]
    return ' '.join(clean_text)

In [46]:
# Apply the stemming function to the 'claim' column and create a new 'processed_claim' column
df['Processed_Claim'] = df['Claim'].apply(stem_text)

In [50]:
# Reorder columns
column_order = ['Nährstoff', 'Claim', 'Processed_Claim', 'Bedingungen']
df = df[column_order]

KeyError: "['Nährstoff'] not in index"

# Save in json file

In [48]:
df.to_json('claims_list.json', orient='records', lines=True, force_ascii=False)

In [49]:
df

,Claim,Processed_Claim,Bedingungen
0,Aktivkohle trägt zur Verringerung übermäßiger ...,Aktivkohle tragen Verringerung übermäßig Blähu...,Die Angabe darf nur für Lebensmittel verwendet...
2,Der Verzehr von Alpha-Cyclodextrin als Bestand...,Verzehr Bestandteil stärkehaltig Mahlzeit trag...,Die Angabe darf verwendet werden für Lebensmit...
4,ALA trägt zur Aufrechterhaltung eines normalen...,ALA tragen Aufrechterhaltung normal Cholesteri...,Die Angabe darf nur für Lebensmittel verwendet...
5,Die Aufnahme von Arabinoxylan als Bestandteil ...,Aufnahme Arabinoxylan Bestandteil Mahlzeit tra...,Die Angabe darf nur für Lebensmittel verwendet...
6,Beta-Glucane tragen zur Aufrechterhaltung eine...,tragen Aufrechterhaltung normal Cholesterinspi...,Die Angabe darf nur für Lebensmittel verwendet...
...,...,...,...
268,Zuckerfreier Kaugummi trägt zur Erhaltung der ...,Zuckerfreier Kaugummi tragen Erhaltung Zahnmin...,Die Angabe darf nur für Kaugummi verwendet wer...
269,Zuckerfreier Kaugummi trägt zur Neutralisierun...,Zuckerfreier Kaugummi tragen Neutralisierung S...,Die Angabe darf nur für Kaugummi verwendet wer...
270,Zuckerfreier Kaugummi trägt zur Verringerung v...,Zuckerfreier Kaugummi tragen Verringerung Mund...,Die Angabe darf nur für Kaugummi verwendet wer...
271,Zuckerfreier Kaugummi mit Carbamid neutralisie...,Zuckerfreier Kaugummi Carbamid neutralisieren ...,Die Angabe darf nur für Kaugummi verwendet wer...
